In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [12]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2023-08-06  16:00  Community Shield  FA Community Shield  Sun   
1          2023-08-11  20:00    Premier League          Matchweek 1  Fri   
2          2023-08-16  22:00         Super Cup       UEFA Super Cup  Wed   
3          2023-08-19  20:00    Premier League          Matchweek 2  Sat   
4          2023-08-27  14:00    Premier League          Matchweek 3  Sun   

                                                ... Standard                \
     Venue Result     GF     GA       Opponent  ...     Dist   FK PK PKatt   
0  Neutral      D  1 (1)  1 (4)        Arsenal  ...      NaN  NaN  0     0   
1     Away      W      3      0        Burnley  ...     13.9  0.0  0     0   
2     Home      D  1 (5)  1 (4)     es Sevilla  ...      NaN  NaN  0     0   
3     Home      W      1      0  Newcastle Utd  ...     17.9  0.0  0     0   
4     Away      W      2      1  Sheffield Utd  ...     17.3  2.0  0     1   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      1.9  1.9    0.12  1.1     1.1        Match Report  
2      NaN  NaN     NaN  NaN     NaN        Match Report  
3      1.0  1.0    0.07  0.0     0.0        Match Report  
4      3.5  2.8    0.10 -1.5    -0.8        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,4-2-3-1,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,8,4,NaN,NaN,0,0
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,Match Report,NaN,17,8,13.9,0.0,0,0
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,4-2-3-1,François Letexier,Match Report,NaN,23,7,NaN,NaN,0,0
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,Match Report,NaN,14,4,17.9,0.0,0,0
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,4-2-3-1,Jarred Gillett,Match Report,NaN,29,9,17.3,2.0,0,1


In [16]:
years = list(range(2022, 2020, -1))
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [29]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

delay_seconds = 1

for year in years:
    data = requests.get(standings_url)
    # Check if the request was successful
    if data.status_code == 200:
        soup = BeautifulSoup(data.text)
        standings_tables = soup.select('table.stats_table')
        if standings_tables: # Check if any tables were found
            standings_table = standings_tables[0]

            links = [l.get("href") for l in standings_table.find_all('a')]
            links = [l for l in links if '/squads/' in l]
            team_urls = [f"https://fbref.com{l}" for l in links]

            previous_season = soup.select("a.prev")[0].get("href")
            standings_url = f"https://fbref.com{previous_season}"

            for team_url in team_urls:
              team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
              data = requests.get(team_url)
              matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
              soup = BeautifulSoup(data.text)
              links = [l.get("href") for l in soup.find_all('a')]
              # Updated match string to reflect the potential change in the website structure
              links = [l for l in links if l and 'all_comps/shooting' in l]
              if links: # Check if any links were found
                data = requests.get(f"https://fbref.com{links[0]}")
                # Added error handling to catch potential issues when reading the shooting table
                try:
                    shooting = pd.read_html(data.text, match="Shooting")[0]
                    shooting.columns = shooting.columns.droplevel()
                except (ValueError, IndexError) as e:
                    print(f"Error reading shooting data for {team_name}: {e}")
                    continue

                try:
                    team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
                except ValueError:
                    continue
                team_data = team_data[team_data["Comp"] == "Premier League"]

                team_data["Season"] = year
                team_data["Team"] = team_name
                all_matches.append(team_data)
                      # ... (rest of your code remains the same)
        else:
            print(f"No standings table found for year {year}. Check the website structure or your selector.")
    else:
        print(f"Request failed for year {year} with status code: {data.status_code}")
        # Consider implementing error handling or retries here
        time.sleep(delay_seconds)  # Add a delay between requests
        delay_seconds *= 2

In [30]:
len(all_matches)


7

In [31]:
match_df = pd.concat(all_matches)

In [32]:
match_df.columns = [c.lower() for c in match_df.columns]

In [33]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2022,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2022,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2022,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2022,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Home,W,5,1,Sheffield Utd,...,Match Report,NaN,19.0,6.0,16.0,0.0,1,1,2022,Newcastle United
47,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Away,W,4,1,Burnley,...,Match Report,NaN,22.0,10.0,17.6,1.0,0,1,2022,Newcastle United
48,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Home,D,1,1,Brighton,...,Match Report,NaN,18.0,7.0,15.8,0.0,0,0,2022,Newcastle United
49,2024-05-15,20:00,Premier League,Matchweek 34,Wed,Away,L,2,3,Manchester Utd,...,Match Report,NaN,21.0,7.0,14.6,0.0,0,0,2022,Newcastle United


In [34]:
match_df.to_csv("matches.csv")